In [1]:
from multiprocessing import Pool
import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm

In [2]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens

In [3]:
"""BPSO"""
def logsig(n): return 1 / (1 + math.exp(-n))
def sign(x): return 1 if x > 0 else (-1 if x!=0 else 0)

def BPSO(Eval_Func,n=20,m_i=200,minf=0,dim=None,prog=True,w1=0.5,c1=1,c2=1,vmax=4,mp=None):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            w1: move rate, default=0.5
            c1,c2: It's are two fixed variables, default=1,1
            vmax: Limit search range of vmax, default=4
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)


    gens=random_search(n,dim)
    pbest=float("-inf") if minf == 0 else float("inf")
    gbest=float("-inf") if minf == 0 else float("inf")
    #vec=3
    #flag=dr
    gens=random_search(n,dim)
    vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    one_vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    zero_vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]

    fit=[float("-inf") if minf == 0 else float("inf") for i in range(n)]
    pbest=dc(fit)
    xpbest=dc(gens)
    #w1=0.5
    if minf==0:
        gbest=max(fit)
        xgbest=gens[fit.index(max(fit))]
    else:
        gbest=min(fit)
        xgbest=gens[fit.index(min(fit))]

    #c1,c2=1,1
    #vmax=4
    gens_dict={tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}
    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)

    for it in miter:
        #w=0.5
        if mp==None:
            for i in range(n):
                if tuple(gens[i]) in gens_dict:
                    score=gens_dict[tuple(gens[i])]
                else:
                    score=estimate(gens[i])
                    gens_dict[tuple(gens[i])]=score
                fit[i]=score
                if fit[i]>pbest[i] if minf==0 else fit[i]<pbest[i]:#max
                    pbest[i]=dc(fit[i])
                    xpbest[i]=dc(gens[i])
                    """multi"""
        else:
            #fit=[float("-inf") if minf == 0 else float("inf") for i in range(n)]
            fit=[gens_dict[tuple(g)]  if tuple(g) in gens_dict  else(float("-inf") if minf == 0 else float("inf")) for g in gens]
            alter_gens=[i for i,g in enumerate(gens) if tuple(g) not in gens_dict]
            #print(len(alter_gens))
            with Pool(mp) as p:
                alter_fit = p.map(estimate,[gens[i] for i in alter_gens])
            # pbest=dc(fit)
            # xpbest=dc(gens)
            for i,f in zip(alter_gens,alter_fit):
                fit[i]=f
                gens_dict[tuple(gens[i])]=f

            #print(alter_fit)
            #print(list(map(estimate,[gens[i] for i in alter_gens])))
            # pbest=dc(fit)
            # xpbest=dc(gens)
            # for i,f in zip(alter_gens,alter_fit):
            #     fit[i]=f
            #print(fit)

            if minf==0:#max
                for i,f in enumerate(fit):
                    if f>pbest[i]:
                        pbest[i]=dc(max(fit))
                        xpbest[i]=dc(gens[fit.index(max(fit))])
            else:
                for i,f in enumerate(fit):
                    if f>pbest[i]:
                        pbest[i]=dc(max(fit))
                        xpbest[i]=dc(gens[fit.index(max(fit))])
        """end"""
        if minf==0:
            gg=max(fit)
            xgg=gens[fit.index(max(fit))]
        else:
            gg=min(fit)
            xgg=gens[fit.index(min(fit))]

        if gbest<gg if minf==0 else gbest>gg:#max
            gbest=dc(gg)
            xgbest=dc(xgg)

        oneadd=[[0 for d in range(dim)] for i in range(n)]
        zeroadd=[[0 for d in range(dim)] for i in range(n)]
        c3=c1*random.random()
        dd3=c2*random.random()
        for i in range(n):
            for j in range(dim):
                if xpbest[i][j]==0:
                    oneadd[i][j]=oneadd[i][j]-c3
                    zeroadd[i][j]=zeroadd[i][j]+c3
                else:
                    oneadd[i][j]=oneadd[i][j]+c3
                    zeroadd[i][j]=zeroadd[i][j]-c3

                if xgbest[j]==0:
                    oneadd[i][j]=oneadd[i][j]-dd3
                    zeroadd[i][j]=zeroadd[i][j]+dd3
                else:
                    oneadd[i][j]=oneadd[i][j]+dd3
                    zeroadd[i][j]=zeroadd[i][j]-dd3

        one_vel=[[w1*_v+_a for _v,_a in zip(ov,oa)] for ov,oa in zip(one_vel,oneadd)]
        zero_vel=[[w1*_v+_a for _v,_a in zip(ov,oa)] for ov,oa in zip(zero_vel,zeroadd)]
        for i in range(n):
            for j in range(dim):
                if abs(vel[i][j]) > vmax:
                    zero_vel[i][j]=vmax*sign(zero_vel[i][j])
                    one_vel[i][j]=vmax*sign(one_vel[i][j])
        for i in range(n):
            for j in range(dim):
                if gens[i][j]==1:
                    vel[i][j]=zero_vel[i][j]
                else:
                    vel[i][j]=one_vel[i][j]
        veln=[[logsig(s[_s]) for _s in range(len(s))] for s in vel]
        temp=[[random.random() for d in range(dim)] for _n in range(n)]
        for i in range(n):
            for j in range(dim):
                if temp[i][j]<veln[i][j]:
                    gens[i][j]= 0 if gens[i][j] ==1 else 1
                else:
                    pass
    print(len(gens_dict))
    return gbest,xgbest,xgbest.count(1)

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import numpy as np

from sklearn import svm
 
SEED = 2018
np.random.seed(SEED)
df_train = pd.read_csv("VoxForge512.csv")
X = np.array(df_train.drop(['label'],axis=1))
y = np.array(df_train['label'])
from sklearn.model_selection import train_test_split
df=df_train
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
# trainX=data
# trainy=label
cross=4
test_size=(1/cross)
trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)

900 201


In [5]:
if __name__ == '__main__':
    from sklearn import svm
    from time import time

    np.random.seed(20)
    tr_d=trainX
    te_d=testX
    tr_l=trainy
    te_l=testy

In [6]:
class Evaluate:#setting class
        def __init__(self):#set train_data,label,test_data,label
            self.train_l=tr_l
            self.train_d=tr_d
            self.test_l=te_l
            self.test_d=te_d
        def evaluate(self,gen):
            """
            Setting of evaluation function.
            Here, the correct answer rate is used.
              anser_label/all_label
            """
            mask=np.array(gen) > 0
            al_data=np.array([al[mask] for al in self.train_d])
            al_test_data=np.array([al[mask] for al in self.test_d])
            #↑masking with [01]sequence list
            res=RandomForestClassifier(n_jobs = -1, verbose = 0, n_estimators=5, criterion='entropy', random_state = 0).fit(al_data,self.train_l).predict(al_test_data)
            return np.count_nonzero(self.test_l==res)/len(self.test_l)
            #↑evaluate with fittness function
        def check_dimentions(self,dim):#check number of all feature
            if dim==None:
                return len(self.train_d[0])
            else:
                return dim

In [15]:
%%time
s,g,l=BPSO(Eval_Func=Evaluate, n=5, m_i=5)

100%|██████████| 5/5 [00:05<00:00,  1.12s/it]

26
CPU times: user 1.14 s, sys: 89.8 ms, total: 1.23 s
Wall time: 5.62 s


In [16]:
l

103

In [17]:
List = []

for i in range(0,len(g)):
    if g[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("VoxForge512.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,label
0,0.003289,0.002809,0.002662,0.002827,0.002435,0.002319,0.000742,0.002926,0.000497,0.000879,...,8.176116,14.609205,2.477992,11.436278,15.037514,14.887147,11.462402,0.009924,0.017622,English
1,0.002735,0.002489,0.002068,0.002175,0.003271,0.001032,0.000193,0.001979,0.000250,0.000249,...,5.993231,9.241147,1.788440,10.301635,19.207247,15.687752,11.421647,0.014316,0.024158,English
2,0.001556,0.001227,0.001315,0.001594,0.000923,0.001047,0.000294,0.001401,0.000330,0.000225,...,8.534367,10.038836,1.958939,11.414680,32.569420,17.805973,11.088083,0.015950,0.028974,English
3,0.007180,0.011593,0.007111,0.014183,0.007152,0.003775,0.002010,0.005047,0.001264,0.002931,...,0.187216,0.475082,0.109862,0.778359,9.625833,1.745809,9.626831,0.010955,0.016978,English
4,0.003632,0.003760,0.002512,0.004510,0.004053,0.001852,0.001037,0.002044,0.000424,0.001479,...,3.347448,5.555333,0.912959,3.941638,10.674464,5.230475,9.285885,0.008553,0.017641,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000754,0.000955,0.000608,0.001347,0.000490,0.000330,0.000327,0.000427,0.000355,0.000458,...,0.011810,0.049385,0.008898,0.045892,0.002370,0.001476,29.929813,0.026434,0.011313,Spanish
896,0.000093,0.000061,0.000111,0.000110,0.000100,0.000101,0.000090,0.000094,0.000126,0.000074,...,0.008861,0.013272,0.002336,0.011562,0.005875,0.001386,9.986907,0.003441,0.002254,Spanish
897,0.000122,0.000083,0.000059,0.000091,0.000062,0.000049,0.000028,0.000041,0.000036,0.000048,...,0.001902,0.006380,0.001357,0.008814,0.000845,0.000601,9.939436,0.005301,0.003221,Spanish
898,0.000005,0.000008,0.000005,0.000006,0.000006,0.000007,0.000017,0.000006,0.000010,0.000014,...,0.003506,0.002084,0.000473,0.003289,0.000557,0.000454,9.965343,0.003344,0.002046,Spanish


In [18]:
df_train1.to_csv('VF_BPSO.csv')